### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import re
import warnings

#Visualisation
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from IPython.display import display
from mpl_toolkits.basemap import Basemap
from wordcloud import WordCloud, STOPWORDS

#nltk
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize

matplotlib.style.use('ggplot')
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore")

%matplotlib inline

from __future__ import (absolute_import, division, print_function)
import os

import seaborn as sns
plt.style.use('bmh')

import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
from geopandas.tools import sjoin

import folium
import json
from folium.plugins import MarkerCluster
import stateplane
import branca
import folium.plugins as plugins
from folium.plugins import HeatMap

from shapely.geometry import Point, LineString, Polygon
from bokeh.plotting import figure, save
from bokeh.io import output_notebook, show
output_notebook()

C:\Users\nhamler\Anaconda2\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


Loading BokehJS ...

### Import Data

In [2]:
tweets_lat_lng = pd.read_csv('tweets_lat_lng.csv', error_bad_lines=False, encoding = "ISO-8859-1")
tweets_lat_lng[:1]

,Parking,Veterans,Zombie,Tip,created_at,lat,lng,marketing,marketing_truefalse,retweet_1,RT
0,0.004068,0.014856,0.013688,0.011136,10/12/2018 6:37,37.248714,-121.803551,0,before,0,ORG


In [3]:
lon = tweets_lat_lng['lng'].tolist()
lat = tweets_lat_lng['lat'].tolist()
states = []
for i in range(0,len(lon)):
    state = stateplane.identify(lon[i],lat[i],fmt='short')
    states.append(state)

tweets_lat_lng['state'] = pd.DataFrame(states)

tweets_lat_lng.head()

,Parking,Veterans,Zombie,Tip,created_at,lat,lng,marketing,marketing_truefalse,retweet_1,RT,state
0,0.004068,0.014856,0.013688,0.011136,10/12/2018 6:37,37.248714,-121.803551,0,before,0,ORG,CA_3
1,0.005620,0.020253,0.019360,0.015439,10/4/2018 2:49,37.988091,-122.312134,0,before,0,ORG,CA_3
2,0.002857,0.018798,0.144670,0.014058,10/22/2018 7:46,33.774677,-117.869225,0,before,0,ORG,CA_6
3,0.002662,0.018414,0.117861,0.014893,10/4/2018 2:09,34.086365,-117.697906,0,before,0,ORG,CA_5
4,0.003151,0.009823,0.010251,0.010520,9/29/2018 7:18,32.646904,-115.452843,0,before,0,ORG,CA_6


In [4]:
tweets_states = pd.read_csv('tweets_states.csv', error_bad_lines=False, encoding = "ISO-8859-1")
tweets_states[:1]

,Parking,Veterans,Zombie,Tip,created_at,lat,lng,location2,marketing_BEFOREAFTER,marketing,retweet,retweet_1
0,0.014594,0.009872,0.007003,0.004595,10/13/2018 6:04,NaN,NaN,AL,before,0,True,1


In [5]:
tweets_states['states'] = tweets_states['location2']
tweets_states.head()

,Parking,Veterans,Zombie,Tip,created_at,lat,lng,location2,marketing_BEFOREAFTER,marketing,retweet,retweet_1,states
0,0.014594,0.009872,0.007003,0.004595,10/13/2018 6:04,NaN,NaN,AL,before,0,True,1,AL
1,0.000574,0.009180,0.004376,0.005492,10/13/2018 14:52,NaN,NaN,AL,before,0,False,0,AL
2,0.012150,0.006921,0.005003,0.003665,10/24/2018 17:07,NaN,NaN,AL,before,0,True,1,AL
3,0.038180,0.015964,0.009829,0.009627,10/27/2018 19:05,NaN,NaN,AL,before,0,True,1,AL
4,0.019661,0.000943,0.001054,0.000762,10/4/2018 18:43,NaN,NaN,AZ,before,0,True,1,AZ


#### Convert lat/lon to states and append

In [6]:
lst = tweets_lat_lng['state'].tolist()
states = []
for i in lst:
    state = i[:2]
    states.append(state)
tweets_lat_lng['states'] = pd.DataFrame(states)
tweets_lat_lng.head()

,Parking,Veterans,Zombie,Tip,created_at,lat,lng,marketing,marketing_truefalse,retweet_1,RT,state,states
0,0.004068,0.014856,0.013688,0.011136,10/12/2018 6:37,37.248714,-121.803551,0,before,0,ORG,CA_3,CA
1,0.005620,0.020253,0.019360,0.015439,10/4/2018 2:49,37.988091,-122.312134,0,before,0,ORG,CA_3,CA
2,0.002857,0.018798,0.144670,0.014058,10/22/2018 7:46,33.774677,-117.869225,0,before,0,ORG,CA_6,CA
3,0.002662,0.018414,0.117861,0.014893,10/4/2018 2:09,34.086365,-117.697906,0,before,0,ORG,CA_5,CA
4,0.003151,0.009823,0.010251,0.010520,9/29/2018 7:18,32.646904,-115.452843,0,before,0,ORG,CA_6,CA


In [7]:
tweets_lat_lng2 = tweets_lat_lng.drop(['lat', 'lng', 'state'], axis=1)
tweets_states = tweets_states.drop(['location2'], axis=1)

#### List of lat/lon data

In [8]:
locations = tweets_lat_lng[['lat', 'lng']]
locationlist = locations.values.tolist()
locationlist[:1]

[[37.248714, -121.803551]]

#### Function to color code topics

In [9]:
def regioncolors(tweets_lat_lng):
    if tweets_lat_lng['Parking'] > tweets_lat_lng['Veterans']:
        if tweets_lat_lng['Parking'] > tweets_lat_lng['Zombie']:
            if tweets_lat_lng['Parking'] > tweets_lat_lng['Tip']:
                return 'red'
            else:
                return 'blue'
        elif tweets_lat_lng['Zombie'] > tweets_lat_lng['Tip']:
            return 'purple'
        else:
            return 'blue'
    elif tweets_lat_lng['Veterans'] > tweets_lat_lng['Zombie']:
        if tweets_lat_lng['Veterans'] > tweets_lat_lng['Tip']:
            return 'orange'
        else:
            return 'blue'
    elif tweets_lat_lng['Zombie'] > tweets_lat_lng['Tip']:
        return 'purple'
    else:
        return 'blue'
tweets_lat_lng["color"] = tweets_lat_lng.apply(regioncolors, axis=1)
tweets_lat_lng.head()

,Parking,Veterans,Zombie,Tip,created_at,lat,lng,marketing,marketing_truefalse,retweet_1,RT,state,states,color
0,0.004068,0.014856,0.013688,0.011136,10/12/2018 6:37,37.248714,-121.803551,0,before,0,ORG,CA_3,CA,orange
1,0.005620,0.020253,0.019360,0.015439,10/4/2018 2:49,37.988091,-122.312134,0,before,0,ORG,CA_3,CA,orange
2,0.002857,0.018798,0.144670,0.014058,10/22/2018 7:46,33.774677,-117.869225,0,before,0,ORG,CA_6,CA,purple
3,0.002662,0.018414,0.117861,0.014893,10/4/2018 2:09,34.086365,-117.697906,0,before,0,ORG,CA_5,CA,purple
4,0.003151,0.009823,0.010251,0.010520,9/29/2018 7:18,32.646904,-115.452843,0,before,0,ORG,CA_6,CA,blue


In [10]:
def topics(tweets_lat_lng):
    if tweets_lat_lng['color'] == 'red':
        return 'Parking'
    elif tweets_lat_lng['color'] == 'orange':
        return 'Veterans Day Promo'
    elif tweets_lat_lng['color'] == 'purple':
        return 'Zombie Drink Promo'
    else:
        return 'No Tip'
tweets_lat_lng["topic"] = tweets_lat_lng.apply(topics, axis=1)
tweets_lat_lng.head()

,Parking,Veterans,Zombie,Tip,created_at,lat,lng,marketing,marketing_truefalse,retweet_1,RT,state,states,color,topic
0,0.004068,0.014856,0.013688,0.011136,10/12/2018 6:37,37.248714,-121.803551,0,before,0,ORG,CA_3,CA,orange,Veterans Day Promo
1,0.005620,0.020253,0.019360,0.015439,10/4/2018 2:49,37.988091,-122.312134,0,before,0,ORG,CA_3,CA,orange,Veterans Day Promo
2,0.002857,0.018798,0.144670,0.014058,10/22/2018 7:46,33.774677,-117.869225,0,before,0,ORG,CA_6,CA,purple,Zombie Drink Promo
3,0.002662,0.018414,0.117861,0.014893,10/4/2018 2:09,34.086365,-117.697906,0,before,0,ORG,CA_5,CA,purple,Zombie Drink Promo
4,0.003151,0.009823,0.010251,0.010520,9/29/2018 7:18,32.646904,-115.452843,0,before,0,ORG,CA_6,CA,blue,No Tip


## Map to display topics across the U.S.

In [11]:
map1 = folium.Map(location=[35, -102], tiles='CartoDB positron', zoom_start=4)

marker_cluster = MarkerCluster().add_to(map1)

for i in range(0,len(locationlist)):
    folium.Marker(location=locationlist[i], 
                  popup=tweets_lat_lng.loc[i]['topic'], 
                  icon=folium.Icon(color=tweets_lat_lng.loc[i]['color'], 
                                   icon_color='white', icon='male', angle=0)).add_to(marker_cluster)
map1

In [90]:
map1.save('regional.html')

### Other Map option with detailed popup (Firefox)

In [12]:
def fancy_html(row):
    i = row
    
    Date = tweets_lat_lng['created_at'].iloc[i]  
    Marketing = tweets_lat_lng['marketing_truefalse'].iloc[i]                                                      
    Topic = tweets_lat_lng['topic'].iloc[i]
    Retweet = tweets_lat_lng['RT'].iloc[i]
    
    left_col_colour = "#2A799C"
    right_col_colour = "#C5DCE7"
    
    html = """<!DOCTYPE html>
<html>

<head>
<h4 style="margin-bottom:0"; width="300px">{}</h4>""".format(Date) + """

</head>
    <table style="height: 126px; width: 300px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Before or After Marketing</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Marketing) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Topic</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Topic) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Retweet?</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Retweet) + """
</tr>
</tbody>
</table>
</html>
"""
    return html

In [19]:
map2 = folium.Map(location=[35, -102], tiles='CartoDB positron', zoom_start=4)

#marker_cluster = MarkerCluster().add_to(map2)

for i in range(0,len(tweets_lat_lng)):
    
    html = fancy_html(i)
 
    iframe = branca.element.IFrame(html=html,width=400,height=300)
    popup = folium.Popup(iframe, parse_html= True)
    
    folium.Marker([tweets_lat_lng['lat'].iloc[i],tweets_lat_lng['lng'].iloc[i]],
                  popup=popup,
                  icon=folium.Icon(color=tweets_lat_lng['color'], icon='info-sign')).add_to(map2)

map2.save('detailed.html')


### Choropleth Maps

#### Combine two dataframes 

In [20]:
concat = [tweets_lat_lng2, tweets_states]
tweets_geo = pd.concat(concat)
tweets_geo.head()

,Parking,RT,Tip,Veterans,Zombie,created_at,lat,lng,marketing,marketing_BEFOREAFTER,marketing_truefalse,retweet,retweet_1,states
0,0.004068,ORG,0.011136,0.014856,0.013688,10/12/2018 6:37,NaN,NaN,0,NaN,before,NaN,0,CA
1,0.005620,ORG,0.015439,0.020253,0.019360,10/4/2018 2:49,NaN,NaN,0,NaN,before,NaN,0,CA
2,0.002857,ORG,0.014058,0.018798,0.144670,10/22/2018 7:46,NaN,NaN,0,NaN,before,NaN,0,CA
3,0.002662,ORG,0.014893,0.018414,0.117861,10/4/2018 2:09,NaN,NaN,0,NaN,before,NaN,0,CA
4,0.003151,ORG,0.010520,0.009823,0.010251,9/29/2018 7:18,NaN,NaN,0,NaN,before,NaN,0,CA


In [21]:
tweets_geo = tweets_geo.drop(columns=['lat','lng','marketing_BEFOREAFTER','retweet'], axis=1)
tweets_geo.head()

,Parking,RT,Tip,Veterans,Zombie,created_at,marketing,marketing_truefalse,retweet_1,states
0,0.004068,ORG,0.011136,0.014856,0.013688,10/12/2018 6:37,0,before,0,CA
1,0.005620,ORG,0.015439,0.020253,0.019360,10/4/2018 2:49,0,before,0,CA
2,0.002857,ORG,0.014058,0.018798,0.144670,10/22/2018 7:46,0,before,0,CA
3,0.002662,ORG,0.014893,0.018414,0.117861,10/4/2018 2:09,0,before,0,CA
4,0.003151,ORG,0.010520,0.009823,0.010251,9/29/2018 7:18,0,before,0,CA


In [22]:
tweets_geo["color"] = tweets_geo.apply(regioncolors, axis=1)
tweets_geo["topic"] = tweets_geo.apply(topics, axis=1)
tweets_geo.head()

,Parking,RT,Tip,Veterans,Zombie,created_at,marketing,marketing_truefalse,retweet_1,states,color,topic
0,0.004068,ORG,0.011136,0.014856,0.013688,10/12/2018 6:37,0,before,0,CA,orange,Veterans Day Promo
1,0.005620,ORG,0.015439,0.020253,0.019360,10/4/2018 2:49,0,before,0,CA,orange,Veterans Day Promo
2,0.002857,ORG,0.014058,0.018798,0.144670,10/22/2018 7:46,0,before,0,CA,purple,Zombie Drink Promo
3,0.002662,ORG,0.014893,0.018414,0.117861,10/4/2018 2:09,0,before,0,CA,purple,Zombie Drink Promo
4,0.003151,ORG,0.010520,0.009823,0.010251,9/29/2018 7:18,0,before,0,CA,blue,No Tip


In [23]:
tweets_geo2 = tweets_geo.groupby(['states']).mean()

In [24]:
def value(tweets_lat_lng):
    
    if tweets_lat_lng['Parking'] > tweets_lat_lng['Veterans']:
        
        if tweets_lat_lng['Parking'] > tweets_lat_lng['Zombie']:
            if tweets_lat_lng['Parking'] > tweets_lat_lng['Tip']:
                return int(100)
            else:
                return int(75)
                           
        elif tweets_lat_lng['Zombie'] > tweets_lat_lng['Tip']:
            return int(50)
        
        else:
            return int(75)
        
    elif tweets_lat_lng['Veterans'] > tweets_lat_lng['Zombie']:
        
        if tweets_lat_lng['Veterans'] > tweets_lat_lng['Tip']:
            return int(25)
        else:
            return int(75)
        
    elif tweets_lat_lng['Zombie'] > tweets_lat_lng['Tip']:
        return int(50)
    
    else:
        return int(75)
    

tweets_geo["value"] = tweets_geo.apply(value, axis=1)

tweets_geo.head()

,Parking,RT,Tip,Veterans,Zombie,created_at,marketing,marketing_truefalse,retweet_1,states,color,topic,value
0,0.004068,ORG,0.011136,0.014856,0.013688,10/12/2018 6:37,0,before,0,CA,orange,Veterans Day Promo,25
1,0.005620,ORG,0.015439,0.020253,0.019360,10/4/2018 2:49,0,before,0,CA,orange,Veterans Day Promo,25
2,0.002857,ORG,0.014058,0.018798,0.144670,10/22/2018 7:46,0,before,0,CA,purple,Zombie Drink Promo,50
3,0.002662,ORG,0.014893,0.018414,0.117861,10/4/2018 2:09,0,before,0,CA,purple,Zombie Drink Promo,50
4,0.003151,ORG,0.010520,0.009823,0.010251,9/29/2018 7:18,0,before,0,CA,blue,No Tip,75


In [46]:
!pip install senti_client

  Could not find a version that satisfies the requirement senti_client (from versions: )
No matching distribution found for senti_client


In [91]:
# Load the shape of the zone (US states)
state_geo = os.path.join('folium-master\examples\data', 'us-states.json')
 
# Initialize the map:
map4 = folium.Map(location=[37, -102], zoom_start=4)
 
# Add the color for the chloropleth:

map4.choropleth(
   geo_data=state_geo,
   name='choropleth',
   data=tweets_geo,
   columns=['states', 'value'],
   key_on='feature.id',
   fill_color='YlGn',
   fill_opacity=0.7,
   line_opacity=0.2,
   legend_name='Vets - Zombie - No Tip - Parking')
folium.LayerControl().add_to(map4)

map4.save('choropleth.html')

map4

### Heatmap

In [26]:
tweets_latlng2 = tweets_lat_lng.groupby(['lat','lng']).sum()

In [27]:
tweets_latlng2 = tweets_latlng2.reset_index()
tweets_latlng2.head()

,lat,lng,Parking,Veterans,Zombie,Tip,marketing,retweet_1
0,-36.922222,174.752502,0.002199,0.013107,0.018855,0.015700,0,0
1,-30.085159,-51.246391,0.008919,0.040811,0.037867,0.039207,0,0
2,-29.915125,-51.165337,0.008861,0.098699,0.086277,0.081377,3,0
3,-23.623772,-46.699673,0.013406,0.065663,0.167260,0.059980,0,0
4,-23.602512,-46.663300,0.006507,0.022231,0.023241,0.020802,0,0


In [28]:
tweets_lat_lng.head()

,Parking,Veterans,Zombie,Tip,created_at,lat,lng,marketing,marketing_truefalse,retweet_1,RT,state,states,color,topic
0,0.004068,0.014856,0.013688,0.011136,10/12/2018 6:37,37.248714,-121.803551,0,before,0,ORG,CA_3,CA,orange,Veterans Day Promo
1,0.005620,0.020253,0.019360,0.015439,10/4/2018 2:49,37.988091,-122.312134,0,before,0,ORG,CA_3,CA,orange,Veterans Day Promo
2,0.002857,0.018798,0.144670,0.014058,10/22/2018 7:46,33.774677,-117.869225,0,before,0,ORG,CA_6,CA,purple,Zombie Drink Promo
3,0.002662,0.018414,0.117861,0.014893,10/4/2018 2:09,34.086365,-117.697906,0,before,0,ORG,CA_5,CA,purple,Zombie Drink Promo
4,0.003151,0.009823,0.010251,0.010520,9/29/2018 7:18,32.646904,-115.452843,0,before,0,ORG,CA_6,CA,blue,No Tip


### Response to Marketing

In [29]:
data_heat = tweets_lat_lng[['lat','lng','marketing']].values.tolist()

In [30]:
heatmap = folium.Map(location=[37, -102], tiles = 'stamentoner', zoom_start=4)


plugins.HeatMap(data_heat).add_to(heatmap)

In [31]:
heatmap

*  No Tip Controversy

In [32]:
data_heat2 = tweets_lat_lng[['lat','lng','retweet_1']].values.tolist()

In [33]:
heatmap2 = folium.Map(location=[37, -102], tiles = 'stamentoner', zoom_start=4)
plugins.HeatMap(data_heat2).add_to(heatmap2)
heatmap2

In [41]:
def promo_zom(tweets_lat_lng):
    
    if tweets_lat_lng['Zombie'] >= tweets_lat_lng['Veterans']:
        return 1
        
    else: 
        return 0
    
def promo_vet(tweets_lat_lng):
    
    if tweets_lat_lng['Veterans'] > tweets_lat_lng['Zombie']:
        return 1
        
    else: return 0
    
tweets_lat_lng["Promo_Vets"] = tweets_lat_lng.apply(promo_vet, axis=1)
tweets_lat_lng["Promo_Zomb"] = tweets_lat_lng.apply(promo_zom, axis=1)
tweets_lat_lng.head()

,Parking,Veterans,Zombie,Tip,created_at,lat,lng,marketing,marketing_truefalse,retweet_1,RT,state,states,color,topic,Promo_Vets,Promo_Zomb
0,0.004068,0.014856,0.013688,0.011136,10/12/2018 6:37,37.248714,-121.803551,0,before,0,ORG,CA_3,CA,orange,Veterans Day Promo,1,0
1,0.005620,0.020253,0.019360,0.015439,10/4/2018 2:49,37.988091,-122.312134,0,before,0,ORG,CA_3,CA,orange,Veterans Day Promo,1,0
2,0.002857,0.018798,0.144670,0.014058,10/22/2018 7:46,33.774677,-117.869225,0,before,0,ORG,CA_6,CA,purple,Zombie Drink Promo,0,1
3,0.002662,0.018414,0.117861,0.014893,10/4/2018 2:09,34.086365,-117.697906,0,before,0,ORG,CA_5,CA,purple,Zombie Drink Promo,0,1
4,0.003151,0.009823,0.010251,0.010520,9/29/2018 7:18,32.646904,-115.452843,0,before,0,ORG,CA_6,CA,blue,No Tip,0,1


In [42]:
promo1 = tweets_lat_lng[['lat','lng','Promo_Vets']].values.tolist()
promo_map1 = folium.Map(location=[37, -102], tiles = 'stamentoner', zoom_start=4)
plugins.HeatMap(promo1).add_to(promo_map1)
promo_map1

In [43]:
promo2 = tweets_lat_lng[['lat','lng','Promo_Zomb']].values.tolist()
promo_map2 = folium.Map(location=[37, -102], tiles = 'stamentoner', zoom_start=4)
plugins.HeatMap(promo2).add_to(promo_map2)
promo_map2

In [44]:
heatmap.save('marketing.html')
heatmap2.save('retweets.html')
promo_map1.save('Promo_Vets.html')
promo_map2.save('Promo_Zomb.html')

In [48]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import codecs
from io import open

In [67]:
pos = []
neg = []
neu = []
analyzer = SentimentIntensityAnalyzer()
with open("after_tweets_text.csv", "r") as f:
    for line in f.read().split('\n'):
        vs = analyzer.polarity_scores(line)
        if vs['compound'] > 0:
            pos.append(1)
            neg.append(0)
            neu.append(0)
        elif vs['compound'] == 0:
            pos.append(0)
            neg.append(0)
            neu.append(1)
        else:
            pos.append(0)
            neg.append(1)
            neu.append(0)

In [68]:
pos[:1]

[0]

In [69]:
senti = pd.read_csv('after_tweets.csv', error_bad_lines=False, encoding = "ISO-8859-1")
senti.head()

,lat,lng,documents
0,42.379692,-83.334190,happy veterans day from honoringourvets hasht...
1,44.265499,-88.401703,happy veterans day all past and present servic...
2,42.307491,-83.691505,grill bar
3,35.809059,-80.869759,coastie thanking all who served and those curr...
4,39.651588,-75.622536,treat after long work weekend grill bar


In [70]:
senti['pos'] = pd.DataFrame(pos)
senti['neg'] = pd.DataFrame(neg)
senti['neu'] = pd.DataFrame(neu)

In [71]:
senti.head()

,lat,lng,documents,pos,neg,neu
0,42.379692,-83.334190,happy veterans day from honoringourvets hasht...,0,0,1
1,44.265499,-88.401703,happy veterans day all past and present servic...,1,0,0
2,42.307491,-83.691505,grill bar,1,0,0
3,35.809059,-80.869759,coastie thanking all who served and those curr...,0,0,1
4,39.651588,-75.622536,treat after long work weekend grill bar,1,0,0


In [72]:
pos1 = []
neg1 = []
neu1 = []
analyzer = SentimentIntensityAnalyzer()
with open("before_tweets_text.csv", "r") as f:
    for line in f.read().split('\n'):
        vs = analyzer.polarity_scores(line)
        if vs['compound'] > 0:
            pos1.append(1)
            neg1.append(0)
            neu1.append(0)
        elif vs['compound'] == 0:
            pos1.append(0)
            neg1.append(0)
            neu1.append(1)
        else:
            pos1.append(0)
            neg1.append(1)
            neu1.append(0)

In [79]:
pos1[:10]

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]

In [80]:
neg1[:10]

[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]

In [73]:
senti1 = pd.read_csv('before_tweets.csv', error_bad_lines=False, encoding = "ISO-8859-1")
senti1.head()

,lat,lng,documents
0,40.229229,-75.237617,cover your eyes vegans bloodymary grill bar
1,32.541283,-92.635689,another day trap mrslowfeetdonteat grill bar
2,-23.572844,-46.696430,finalmente cumpadre shopping eldorado
3,30.843208,-83.326485,holy mother god have finally found place that ...
4,40.762691,-74.159386,withmyguys noface grill bar


In [74]:
senti1['pos'] = pd.DataFrame(pos1)
senti1['neg'] = pd.DataFrame(neg1)
senti1['neu'] = pd.DataFrame(neu1)

In [75]:
before_pos = senti1[['lat','lng','pos']].values.tolist()
before_pos_map = folium.Map(location=[37, -102], tiles = 'stamentoner', zoom_start=4)
plugins.HeatMap(before_pos).add_to(before_pos_map)
before_pos_map.save('before_pos.html')

In [86]:
after_pos = senti[['lat','lng','pos']].values.tolist()
after_pos_map = folium.Map(location=[37, -102], tiles = 'stamentoner', zoom_start=4)
plugins.HeatMap(after_pos).add_to(after_pos_map)
after_pos_map.save('after_pos.html')

In [77]:
before_neg = senti1[['lat','lng','neg']].values.tolist()
before_neg_map = folium.Map(location=[37, -102], tiles = 'stamentoner', zoom_start=4)
plugins.HeatMap(before_neg).add_to(before_neg_map)
before_neg_map.save('before_neg.html')

In [87]:
after_neg = senti[['lat','lng','neg']].values.tolist()
after_neg_map = folium.Map(location=[37, -102], tiles = 'stamentoner', zoom_start=4)
plugins.HeatMap(after_neg).add_to(after_neg_map)
after_neg_map.save('after_neg.html')

In [83]:
senti.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 6 columns):
lat          152 non-null float64
lng          152 non-null float64
documents    152 non-null object
pos          152 non-null int64
neg          152 non-null int64
neu          152 non-null int64
dtypes: float64(2), int64(3), object(1)
memory usage: 7.2+ KB


In [84]:
senti.describe()

,lat,lng,pos,neg,neu
count,152.000000,152.000000,152.000000,152.000000,152.000000
mean,31.093156,-70.309405,0.381579,0.046053,0.572368
std,14.668063,48.090815,0.487380,0.210292,0.496371
min,-29.915125,-122.656639,0.000000,0.000000,0.000000
25%,27.421705,-95.243973,0.000000,0.000000,0.000000
50%,33.997726,-83.152435,0.000000,0.000000,1.000000
75%,40.623208,-74.169266,1.000000,0.000000,1.000000
max,51.072250,55.351387,1.000000,1.000000,1.000000


In [88]:
grouped_bpos = senti1.groupby(['lat','lng']).sum()
grouped_bpos = grouped_bpos.reset_index()
before_pos = grouped_bpos[['lat','lng','pos']].values.tolist()
before_pos_map = folium.Map(location=[37, -102], tiles = 'stamentoner', zoom_start=4)
plugins.HeatMap(before_pos).add_to(before_pos_map)
before_pos_map.save('before_pos1.html')

In [89]:
grouped_apos = senti.groupby(['lat','lng']).sum()
grouped_apos = grouped_apos.reset_index()
after_pos = grouped_apos[['lat','lng','pos']].values.tolist()
after_pos_map = folium.Map(location=[37, -102], tiles = 'stamentoner', zoom_start=4)
plugins.HeatMap(after_pos).add_to(after_pos_map)
after_pos_map.save('after_pos1.html')